In [ ]:
# 这是一个用来得到ec的精细话数据的脚本

# 数据类型
1. vis 
2. uv_10
3. tmax
4. tmin
5. tcc 
6. t2
7. skt
8. lsp(大尺度降水)
9. conv(对流降水)


In [2]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
import netCDF4 
import h5py     
# 测试数据的可用性
def read_data():
        file_path = "/home/liyuan3970/Data/My_Git/2022041700/"        
        file_name = file_path +"ecfine.I2022041700.000.F2022041700.nc"
        f = netCDF4.Dataset(file_name)
        data_xr_nc = f.variables['t']
        lat = f.variables['latP'][:]
        lon = f.variables['lonP'][:]
        time = f.variables['time'][:]
          
        # return lat,lon,time,data_xr_nc
read_data()


In [3]:
# 面向过程的编程
import xarray as xr
file_path = "/home/liyuan3970/Data/My_Git/2022041700/*.nc" 
f = xr.open_mfdataset(file_path, parallel=True)



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,
/opt/conda/lib/python3.7/site-packages/xarray/core/alignment.py:298: FutureWarning: Index.__or__ opera

In [14]:
# 读取固定点的数据
#选择固定经纬度范围的数据
# sub=ff.sel(lon=slice(105,120),lat=slice(15,30))
t =f.t
t.sel(lev=1000,lonP=125,latP=27.5,method='nearest').data


dask.array<getitem, shape=(53,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>

In [18]:
# 线性插值的核心
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.interpolate import interp1d

x = [1,2,3,4,5,6,7,8,9,10]
y = [1,2,3,4,5,6,7,8,9,10] # 对应没每个点的值
f1=interp1d(x,y,kind='linear')#,kind='cubic'
f1=interp1d(x,y,kind='cubic')#,kind='cubic'
x_pre = [1.1,2.2,3.3,4.4,5.5,6.6,7.7,8.8,9.9]

y_pre = f1(x_pre)

y_pre


array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [21]:
# 数据结构的设计
data = {
    '淡竹':{
        'value1':[53],
        'value2':[53],
        'value3':[53]
    },
    '洪家':{
        'value1':[53],
        'value2':[53],
        'value3':[53]
    }
    }

In [16]:
import xarray as xr
class ec_data_point:
    def __init__(self, select_time,select_type,select_lat,select_lon):
        pass
    def read_data(self):
        file_path = "/home/liyuan3970/Data/My_Git/2022041700/*.nc" 
        print(select_time)
        # f = xr.open_mfdataset(file_path,combine='by_coords',concat_dim=None,data_vars='t', parallel=True)
        f = xr.open_mfdataset(file_path, parallel=True)
        print(f)

        
        
    def render_tojson():
        pass
        
select_time,select_type,select_lat,select_lon = '2022041700','t',27.5,125.7
ec_worker = ec_data_point(select_time,select_type,select_lat,select_lon)        
imd_list = ec_worker.read_data()

2022041700


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/xarray/backends/api.py:941: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,
/opt/conda/lib/python3.7/site-packages/xarray/core/alignment.py:298: FutureWarning: Index.__or__ operating as a s

<xarray.Dataset>
Dimensions:  (latP: 241, latS: 481, lev: 15, lonP: 361, lonS: 721, time: 53)
Coordinates:
  * latS     (latS) float32 60.0 59.875 59.75 59.625 ... 0.375 0.25 0.125 0.0
  * latP     (latP) float32 60.0 59.75 59.5 59.25 59.0 ... 1.0 0.75 0.5 0.25 0.0
  * lonP     (lonP) float32 60.0 60.25 60.5 60.75 ... 149.25 149.5 149.75 150.0
  * lonS     (lonS) float32 60.0 60.125 60.25 60.375 ... 149.75 149.875 150.0
  * lev      (lev) int64 50 100 150 200 250 300 400 ... 700 850 900 925 950 1000
  * time     (time) datetime64[ns] 2022-04-17 2022-04-17T03:00:00 ... 2022-04-27
Data variables:
    vis      (time, latS, lonS) float32 dask.array<chunksize=(1, 481, 721), meta=np.ndarray>
    deg0l    (time, latS, lonS) float32 dask.array<chunksize=(1, 481, 721), meta=np.ndarray>
    fg103    (time, latS, lonS) float32 dask.array<chunksize=(1, 481, 721), meta=np.ndarray>
    sden     (time, latS, lonS) float32 dask.array<chunksize=(1, 481, 721), meta=np.ndarray>
    cape     (time, latS, 